In [18]:
# %tensorflow_version 1.x
import csv
import re
import pandas as pd
import os
import numpy as np
import cv2
# import tensorflow as tf
from tqdm import tqdm_notebook

#from keras.applications.xception import Xception
# from tensorflow.keras.applications import VGG16, EfficientNetB0

In [19]:
!pip3 install --quiet tensorflow-text

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # Imports TF ops for preprocessing.

In [20]:
class VideoToFrames:
    def __init__(self, video_path, no_frames=15):
        self.reader = cv2.VideoCapture(video_path)

        self.no_frames = no_frames
        self.total_frames = int(self.reader.get(cv2.CAP_PROP_FRAME_COUNT))
        if self.total_frames != 0:
            self.interval = int(self.total_frames/self.no_frames)

    def convert(self):
        frames = []
        frame_number = 1
        while len(frames) < self.no_frames:
            self.reader.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            ret, frame = self.reader.read()

            try:
                if frame == None:
                    break
            except ValueError:
                pass

            frames.append(cv2.resize(frame, (48, 48)))
            frame_number += self.interval

        return frames

In [21]:
# Define some sentences to feed into the model
sentences = [
  "Fuck you, my child is completely fine aaron",
  "Here We Go Then, You And I is a 1999 album by Norwegian pop artist Morten Abel. It was Abel's second CD as a solo artist.",
  "The album went straight to number one on the Norwegian album chart, and sold to double platinum.",
  "Ceylon spinach is a common name for several plants and may refer to: Basella alba Talinum fruticosum",
  "A solar eclipse occurs when the Moon passes between Earth and the Sun, thereby totally or partly obscuring the image of the Sun for a viewer on Earth.",
  "A partial solar eclipse occurs in the polar regions of the Earth when the center of the Moon's shadow misses the Earth.",
]

# Load the BERT encoder and preprocessing models
preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert = hub.load('https://tfhub.dev/google/experts/bert/wiki_books/sst2/2')

# preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
# bert = hub.load('https://tfhub.dev/google/experts/bert/wiki_books/mnli/2')

In [22]:
# Convert the sentences to bert inputs
bert_inputs = preprocess(sentences)

# Feed the inputs to the model to get the pooled and sequence outputs
bert_outputs = bert(bert_inputs, training=False)
pooled_output = bert_outputs['pooled_output']
sequence_output = bert_outputs['sequence_output']

print('\nSentences:')
print(sentences)
print('\nPooled output:')
print(pooled_output)
print('\nSequence output:')
print(sequence_output)


Sentences:
['Fuck you, my child is completely fine aaron', "Here We Go Then, You And I is a 1999 album by Norwegian pop artist Morten Abel. It was Abel's second CD as a solo artist.", 'The album went straight to number one on the Norwegian album chart, and sold to double platinum.', 'Ceylon spinach is a common name for several plants and may refer to: Basella alba Talinum fruticosum', 'A solar eclipse occurs when the Moon passes between Earth and the Sun, thereby totally or partly obscuring the image of the Sun for a viewer on Earth.', "A partial solar eclipse occurs in the polar regions of the Earth when the center of the Moon's shadow misses the Earth."]

Pooled output:
tf.Tensor(
[[-0.05643273 -0.8124131   0.635723   ...  0.71955854  0.19358702
  -0.15669791]
 [ 0.45328495  0.40238184  0.6819164  ...  0.87728333  0.8688863
  -0.4735435 ]
 [-0.5207473  -0.08203781  0.71232826 ...  0.58035153  0.8973055
  -0.7298693 ]
 [ 0.57188773  0.16353984 -0.6850588  ...  0.5572128  -0.8921096
 

In [23]:
image_feature_extractor = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
                   trainable=False),  # Can be True, see below.
])
image_feature_extractor.build([None, 48, 48, 3])
image_feature_extractor.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              4049564   
Total params: 4,049,564
Trainable params: 0
Non-trainable params: 4,049,564
_________________________________________________________________


In [24]:
video_dir = '/content/drive/MyDrive/dataset/videoFolder'
video_shape = (30, 512)

In [25]:
# f = open('/content/drive/MyDrive/dataset/postwise_comment_video_embed.csv', 'w')
# writer = csv.writer(f)
# writer.writerow(['link', 'features'])

In [26]:
df = pd.read_csv('/content/drive/MyDrive/dataset/vine_labeled_cyberbullying_data.csv')
df.head()
url_to_postid = pd.read_csv('/content/drive/MyDrive/dataset/urls_to_postids.txt')

In [27]:
len(df['videolink']), len(url_to_postid)

(970, 970)

In [28]:
def link_to_name(link):
    #print(list(url_to_postid[url_to_postid.videolink == link].postid)[0])
    return list(url_to_postid[url_to_postid.videolink == link].postid)[0]

In [29]:
# def image_to_features(images):
#     features = image_feature_extractor.predict(np.array(images))
#     images = []
#     for image in features:
#       images.append(image.reshape(2304*2*2))
#     return np.array(images)

In [31]:
count = 0
error = 0
with open('/content/drive/MyDrive/dataset/postwise_comment_video_embeds_v3.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(['_unit_id', 'label', 'comment_embedding', 'video_embedding'])
for i in tqdm_notebook(range(len(df))):
    videoname = link_to_name(df['videolink'][i])
    converter = VideoToFrames(video_path=os.path.join(video_dir, '{}.mp4'.format(videoname)), no_frames=15)

    frames = converter.convert()
    if(len(frames)==0):
      frames = np.zeros((15, 48, 48, 3))
      # count = count + 1
      # print("No video", count)
      # continue
    
    features = np.array(image_feature_extractor.predict(np.array(frames))).tolist()
    # print(len(features), len(features[0]))
    comments = []
    for j in range(1,100):
      comm=df['column{}'.format(j)][i]
      comm = [x.strip() for x in re.findall('\:\:.*?(.*)\(created', comm, re.MULTILINE | re.DOTALL)]
      if(len(comm)==0):
        continue;
      comm = comm[0]
      if(comm in comments):
        continue;
      comments.append(comm)
      #print(comm)
      if(len(comments)==15):
        break;
    if(len(comments)!=15):
      count=count+1
      print('No text', count)
      continue;
    # Convert the sentences to bert inputs
    bert_inputs = preprocess(comments)
    # Feed the inputs to the model to get the pooled and sequence outputs
    bert_outputs = bert(bert_inputs, training=False)
    comment_embeddings = np.array(bert_outputs['pooled_output']).tolist()

    #comment_embeddings = np.array(sbert_model.encode(comments)).tolist()
    label=1
    if(df['question2'][i]=='noneBll'):
      label=0
    with open('/content/drive/MyDrive/dataset/postwise_comment_video_embeds_v3.csv','a') as file:
      writer = csv.writer(file)
      # print(int(df['_unit_id'][i]), str(label))
      writer.writerow([int(df['_unit_id'][i]), str(label), comment_embeddings, features])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


No text 1
No text 2
No text 3
No text 4
No text 5
No text 6
No text 7
No text 8
No text 9
No text 10
No text 11
No text 12
No text 13
No text 14
No text 15
No text 16
No text 17



In [ ]:
!python train.py -m"bilstm"

In [32]:
df = pd.read_csv('/content/drive/MyDrive/dataset/postwise_comment_video_embeds_v3.csv')
df

,_unit_id,label,comment_embedding,video_embedding
0,698432761,0,"[[-0.2728123962879181, -0.8870245814323425, -0...","[[-0.0, 57.49569320678711, 27.893054962158203,..."
1,698432762,0,"[[0.1890413463115692, -0.5301359295845032, 0.5...","[[-0.09052328765392303, 0.16137035191059113, -..."
2,698432763,0,"[[0.49692460894584656, -0.26320603489875793, 0...","[[-0.258944571018219, -0.17833641171455383, -0..."
3,698432764,0,"[[-0.13861845433712006, -0.7796138525009155, 0...","[[-0.0, 439.99163818359375, 1842.6995849609375..."
4,698432765,1,"[[-0.3295425474643707, -0.2916281223297119, -0...","[[-0.258944571018219, -0.17833641171455383, -0..."
...,...,...,...,...
948,698447713,1,"[[-0.4126042425632477, -0.2747970521450043, 0....","[[-0.0, 1228.775634765625, 2014.6953125, 203.7..."
949,698447782,1,"[[0.38253042101860046, -0.13010577857494354, -...","[[-0.0, 385.495361328125, -0.0, -0.0, -0.0, -0..."
950,698447845,0,"[[-0.15763774514198303, -0.7029466032981873, -...","[[-0.0, -2.2642236236401715e-16, -1.2854285635..."
951,698447928,0,"[[-0.40601250529289246, -0.4909649193286896, 0...","[[-0.258944571018219, -0.17833641171455383, -0..."
